# Amazon Textract - 收据

在这个示例中，我们使用Amazon Textract的`AnalyzeExpense` API来使用Textract的预构建分析来解析收据。此API提供了一种结构化的方式来提取关键信息，如金额、日期、付款方式等。

Amazon Textract还有许多其他用于特定用例的API，例如从表格中提取数据的`AnalyzeDocument` API，以及基础的`DetectDocumentText` API（用于OCR）。

`AnalyzeExpense` API文档在[这里](https://docs.aws.amazon.com/textract/latest/dg/how-it-works-analyzing.html)。

In [ ]:
%pip install --upgrade --quiet  amazon-textract-caller langchain-openai tiktoken

要使用此样本，您需要安装boto3 AWS SDK并配置凭证。请参阅[这里](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#configuration)获取更多详情。

In [ ]:
from langchain_community.document_loaders import AmazonTextractReceiptLoader

## 使用本地文件的示例

In [ ]:
loader = AmazonTextractReceiptLoader("example_data/amazon_receipt.jpg")

您可以直接传递文件路径，如上所示，或指定区域、客户端和Textract配置，如下所示。

In [ ]:
import boto3
from textractor import Textractor

textract_client = boto3.client("textract", region_name="us-east-2")
textractor = Textractor(client=textract_client)

loader = AmazonTextractReceiptLoader(
    "example_data/amazon_receipt.jpg", textractor=textractor
)

## 使用S3上的文件的示例

In [ ]:
s3_file = "s3://amazon-textract-public-content/langchain/simple_receipt.png"

# 我们知道此示例位于us-east-2区域的S3存储桶中
# 所以我们需要使用同一区域的textract客户端
textract_client = boto3.client("textract", region_name="us-east-2")
textractor = Textractor(client=textract_client)

loader = AmazonTextractReceiptLoader(s3_file, textractor=textractor)

### 加载从Amazon Textract解析的收据数据并将其转换为LangChain文档

In [ ]:
documents = loader.load()

In [ ]:
print(documents[0].page_content)

## 查询LangChain文档

有了收据的LangChain文档，我们可以现在使用LangChain问它一些问题！

In [ ]:
# 或在环境中直接设置OpenAI密钥
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

chain = load_qa_chain(llm=OpenAI(), chain_type="stuff")
query = ["发票总额是多少？"]
chain.run(input_documents=documents, question=query)